# RankNET (RECOLA)

In [1]:
import os
import gc 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.nn import leaky_relu
from sklearn.metrics import accuracy_score
from scipy.stats import pearsonr, kendalltau
from sklearn.model_selection import GroupKFold
from tensorflow.keras import layers, Model, Input

In [2]:
project_directory = r'C:\Users\marco\OneDrive\Desktop\Final Year Project'
os.chdir(project_directory)
base_dir = os.getcwd() 

- Evaluation for arousal

In [3]:
def pairwise_transformation(X, Y, groups):
    transformed_data = []
    transformed_labels = []
    transformed_groups = []
    for i in range(len(X)):
        for j in range(i+1, len(X)):
            xi, xj = X[i], X[j]
            yi, yj = Y[i], Y[j]
            if np.abs(yi - yj) > 0.1:
                pair1 = np.concatenate([xi, xj])
                pair2 = np.concatenate([xj, xi])
                transformed_data.append(pair1)
                transformed_data.append(pair2)

                transformed_groups.append(groups[i])
                transformed_groups.append(groups[i])

                if yi > yj:
                    transformed_labels.append(1)
                    transformed_labels.append(0)
                else:
                    transformed_labels.append(0)
                    transformed_labels.append(1)

    return np.array(transformed_data), np.array(transformed_labels), np.array(transformed_groups)


class RankNet(Model):
    def __init__(self):
        super().__init__()
        self.dense = [layers.Dense(16, activation=leaky_relu), layers.Dense(8, activation=leaky_relu)]
        self.o = layers.Dense(1, activation='linear')
        self.oi_minus_oj = layers.Subtract()

    def call(self, input_tensor):
        half = input_tensor.shape[-1] // 2
        xi, xj = input_tensor[:, :half], input_tensor[:, half:]
        densei = self.dense[0](xi)
        densej = self.dense[0](xj)
        for dense in self.dense[1:]:
            densei = dense(densei)
            densej = dense(densej)
        oi = self.o(densei)
        oj = self.o(densej)
        oij = self.oi_minus_oj([oi, oj])
        output = layers.Activation('sigmoid')(oij)
        return output

input_file = os.path.join('RECOLA Ranking Algorithms', 'RECOLA_Intervals_Data', 'ArousalValenceTimeSeries_Quartiles.csv')
df = pd.read_csv(input_file)

def concordance_correlation_coefficient(y_true, y_pred):
    cor = np.corrcoef(y_true, y_pred)[0][1]
    mean_true, mean_pred = np.mean(y_true), np.mean(y_pred)
    var_true, var_pred = np.var(y_true), np.var(y_pred)
    sd_true, sd_pred = np.std(y_true), np.std(y_pred)
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred)**2
    ccc = numerator / denominator
    return ccc

def pearson_correlation_coefficient(y_true, y_pred):
    pcc, _ = pearsonr(y_true, y_pred)
    return pcc

def kendalls_tau_coefficient(y_true, y_pred):
    tau, _ = kendalltau(y_true, y_pred)
    return tau

# Evaluation function 
def evaluate_ranknet_performance(ranknet_model, X_transformed, Y, groups):
    evaluation_results = []
    unique_groups = np.unique(groups)
    for group_id in unique_groups:
        idx = groups == group_id
        group_features = X_transformed[idx]
        group_labels = Y[idx]

        mean_features = group_features.mean(axis=0)
        normalized_features = group_features - mean_features
   
        predicted_scores = ranknet_model.predict(normalized_features).flatten()
   
        ccc_value = concordance_correlation_coefficient(group_labels, predicted_scores)
        pcc_value = pearson_correlation_coefficient(group_labels, predicted_scores)
        kendall_tau_value = kendalls_tau_coefficient(group_labels, predicted_scores)
   
        evaluation_results.append({'GroupID': group_id, 'CCC': ccc_value, 'PCC': pcc_value, 'KendallTau': kendall_tau_value})
    return pd.DataFrame(evaluation_results)

evaluation_results = []  

# Initialize the RankNet model
ranknet_model = RankNet()
ranknet_model.compile(optimizer='adam', loss='binary_crossentropy')

excluded_features = [
        'VIDEO_40_LLD_AU1', 'VIDEO_40_LLD_AU2', 'VIDEO_40_LLD_AU4', 'VIDEO_40_LLD_AU5', 'VIDEO_40_LLD_AU6', 'VIDEO_40_LLD_AU7', 'VIDEO_40_LLD_AU9', 'VIDEO_40_LLD_AU11', 'VIDEO_40_LLD_AU12', 'VIDEO_40_LLD_AU15',
        'VIDEO_40_LLD_AU17', 'VIDEO_40_LLD_AU20', 'VIDEO_40_LLD_AU23', 'VIDEO_40_LLD_AU24', 'VIDEO_40_LLD_AU25', 'VIDEO_40_LLD_Yaw', 'VIDEO_40_LLD_Pitch', 'VIDEO_40_LLD_Roll', 'VIDEO_40_LLD_Opt_mean',
        'VIDEO_40_LLD_Opt_std', 'VIDEO_40_LLD_AU1_delta', 'VIDEO_40_LLD_AU2_delta', 'VIDEO_40_LLD_AU4_delta', 'VIDEO_40_LLD_AU5_delta', 'VIDEO_40_LLD_AU6_delta', 'VIDEO_40_LLD_AU7_delta', 'VIDEO_40_LLD_AU9_delta',
        'VIDEO_40_LLD_AU11_delta', 'VIDEO_40_LLD_AU12_delta', 'VIDEO_40_LLD_AU15_delta', 'VIDEO_40_LLD_AU17_delta', 'VIDEO_40_LLD_AU20_delta', 'VIDEO_40_LLD_AU23_delta', 'VIDEO_40_LLD_AU24_delta', 'VIDEO_40_LLD_AU25_delta',
        'VIDEO_40_LLD_Yaw_delta', 'VIDEO_40_LLD_Pitch_delta', 'VIDEO_40_LLD_Roll_delta', 'VIDEO_40_LLD_Opt_mean_delta', 'VIDEO_40_LLD_Opt_std_delta', 'Face_detection_probability', 'ECG_54_LLD_ECG_HR', 'ECG_54_LLD_ECG_HRV',
        'ECG_54_LLD_ECG_zcr', 'ECG_54_LLD_ECG_FFT_1', 'ECG_54_LLD_ECG_FFT_2', 'ECG_54_LLD_ECG_FFT_3', 'ECG_54_LLD_ECG_FFT_4', 'ECG_54_LLD_ECG_FFT_5', 'ECG_54_LLD_ECG_FFT_6', 'ECG_54_LLD_ECG_FFT_7', 'ECG_54_LLD_ECG_FFT_8',
        'ECG_54_LLD_ECG_FFT_9', 'ECG_54_LLD_ECG_FFT_10', 'ECG_54_LLD_ECG_FFT_11', 'ECG_54_LLD_ECG_FFT_12', 'ECG_54_LLD_ECG_FFT_entropy', 'ECG_54_LLD_ECG_FFT_mean_frequency', 'ECG_54_LLD_ECG_FFT_slope', 'ECG_54_LLD_ECG_mean',
        'ECG_54_LLD_ECG_std', 'ECG_54_LLD_ECG_kurtosis', 'ECG_54_LLD_ECG_skewness', 'ECG_54_LLD_ECG_NSImn', 'ECG_54_LLD_ECG_NLDmn', 'ECG_54_LLD_ECG_VLF', 'ECG_54_LLD_ECG_LF', 'ECG_54_LLD_ECG_HF', 'ECG_54_LLD_ECG_LFHF', 'ECG_54_LLD_ECG_zcr_delta',
        'ECG_54_LLD_ECG_FFT_1_delta', 'ECG_54_LLD_ECG_FFT_2_delta', 'ECG_54_LLD_ECG_FFT_3_delta', 'ECG_54_LLD_ECG_FFT_4_delta', 'ECG_54_LLD_ECG_FFT_5_delta', 'ECG_54_LLD_ECG_FFT_6_delta', 'ECG_54_LLD_ECG_FFT_7_delta', 'ECG_54_LLD_ECG_FFT_8_delta',
        'ECG_54_LLD_ECG_FFT_9_delta', 'ECG_54_LLD_ECG_FFT_10_delta', 'ECG_54_LLD_ECG_FFT_11_delta', 'ECG_54_LLD_ECG_FFT_12_delta', 'ECG_54_LLD_ECG_FFT_entropy_delta', 'ECG_54_LLD_ECG_FFT_mean_frequency_delta', 'ECG_54_LLD_ECG_FFT_slope_delta', 'ECG_54_LLD_ECG_mean_delta',
        'ECG_54_LLD_ECG_std_delta', 'ECG_54_LLD_ECG_kurtosis_delta', 'ECG_54_LLD_ECG_skewness_delta', 'ECG_54_LLD_ECG_NSImn_delta', 'ECG_54_LLD_ECG_NLDmn_delta', 'ECG_54_LLD_ECG_VLF_delta', 'ECG_54_LLD_ECG_LF_delta', 'ECG_54_LLD_ECG_HF_delta', 'ECG_54_LLD_ECG_LFHF_delta',
        'EDA_62_LLD_time_code', 'EDA_62_LLD_EDA_slope', 'EDA_62_LLD_EDA_std', 'EDA_62_LLD_SCR_FFT_entropy', 'EDA_62_LLD_SCR_FFT_mean_frequency', 'EDA_62_LLD_EDA_mean', 'EDA_62_LLD_EDA_meanD', 'EDA_62_LLD_EDA_meanDneg', 'EDA_62_LLD_EDA_prop', 'EDA_62_LLD_EDA_Xbound', 'EDA_62_LLD_EDA_kurtosis',
        'EDA_62_LLD_EDA_skewness', 'EDA_62_LLD_EDA_NSImn', 'EDA_62_LLD_EDA_NLDmn', 'EDA_62_LLD_SCL_mean', 'EDA_62_LLD_SCL_meanD', 'EDA_62_LLD_SCL_meanDneg', 'EDA_62_LLD_SCL_prop', 'EDA_62_LLD_SCL_Xbound', 'EDA_62_LLD_SCL_kurtosis', 'EDA_62_LLD_SCL_skewness', 'EDA_62_LLD_SCL_NSImn',
        'EDA_62_LLD_SCL_NLDmn', 'EDA_62_LLD_SCR_mean', 'EDA_62_LLD_SCR_meanD', 'EDA_62_LLD_SCR_meanDneg', 'EDA_62_LLD_SCR_prop', 'EDA_62_LLD_SCR_Xbound', 'EDA_62_LLD_SCR_kurtosis', 'EDA_62_LLD_SCR_skewness', 'EDA_62_LLD_SCR_NSImn', 'EDA_62_LLD_SCR_NLDmn', 'EDA_62_LLD_EDA_slope_delta',
        'EDA_62_LLD_EDA_std_delta', 'EDA_62_LLD_SCR_FFT_entropy_delta', 'EDA_62_LLD_SCR_FFT_mean_frequency_delta', 'EDA_62_LLD_EDA_mean_delta', 'EDA_62_LLD_EDA_meanD_delta', 'EDA_62_LLD_EDA_meanDneg_delta', 'EDA_62_LLD_EDA_prop_delta', 'EDA_62_LLD_EDA_Xbound_delta', 'EDA_62_LLD_EDA_kurtosis_delta',
        'EDA_62_LLD_EDA_skewness_delta', 'EDA_62_LLD_EDA_NSImn_delta', 'EDA_62_LLD_EDA_NLDmn_delta', 'EDA_62_LLD_SCL_mean_delta', 'EDA_62_LLD_SCL_meanD_delta', 'EDA_62_LLD_SCL_meanDneg_delta', 'EDA_62_LLD_SCL_prop_delta', 'EDA_62_LLD_SCL_Xbound_delta', 'EDA_62_LLD_SCL_kurtosis_delta', 'EDA_62_LLD_SCL_skewness_delta',
        'EDA_62_LLD_SCL_NSImn_delta', 'EDA_62_LLD_SCL_NLDmn_delta', 'EDA_62_LLD_SCR_mean_delta', 'EDA_62_LLD_SCR_meanD_delta', 'EDA_62_LLD_SCR_meanDneg_delta', 'EDA_62_LLD_SCR_prop_delta', 'EDA_62_LLD_SCR_Xbound_delta', 'EDA_62_LLD_SCR_kurtosis_delta', 'EDA_62_LLD_SCR_skewness_delta', 'EDA_62_LLD_SCR_NSImn_delta', 'EDA_62_LLD_SCR_NLDmn_delta'

    ]

feature_columns = [col for col in df.columns if col not in excluded_features + ['participant_id','median_arousal', 'median_valence', 'time_window', 'arousal_quartile', 'valence_quartile']]

targets = ['arousal_quartile']

# Initialize GroupKFold
group_kfold = GroupKFold(n_splits=5)

# Process for arousal_quartile
for target in targets:
    print(f"\nProcessing for {target}:")
    X = df[feature_columns].values
    Y = df[target].values
    groups = df['participant_id'].values
    for fold, (train_index, test_index) in enumerate(group_kfold.split(X, Y, groups=groups)):
        print(f"\nFold {fold+1}/{group_kfold.n_splits} for {target}:")
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        groups_train, groups_test = groups[train_index], groups[test_index]

        # Transform both training and testing data along with groups
        X_train_transformed, Y_train_transformed, groups_train_transformed = pairwise_transformation(X_train, Y_train, groups_train)
        X_test_transformed, Y_test_transformed, groups_test_transformed = pairwise_transformation(X_test, Y_test, groups_test)

        ranknet_model.fit(X_train_transformed, Y_train_transformed, epochs=2, verbose=1)

        evaluation_df = evaluate_ranknet_performance(ranknet_model, X_test_transformed, Y_test_transformed, groups_test_transformed)
        evaluation_results.append(evaluation_df)
        del X_train_transformed, Y_train_transformed
        del X_test_transformed, Y_test_transformed     
        gc.collect()

# Combine and save evaluation results
combined_results_df = pd.concat(evaluation_results, ignore_index=True)

combined_evaluation_results_file = os.path.join('RECOLA Ranking Algorithms','Evaluation','RankNET', 'ranknet_evaluation_results_arousal.csv')
combined_results_df.to_csv(combined_evaluation_results_file, index=False)

print("Individual performance evaluation arousal results saved.")


Processing for arousal_quartile:

Fold 1/5 for arousal_quartile:
Epoch 1/2
116133/116133 ━━━━━━━━━━━━━━━━━━━━ 152s 1ms/step - loss: 234.1577
Epoch 2/2
116133/116133 ━━━━━━━━━━━━━━━━━━━━ 132s 1ms/step - loss: 3.6597
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 2s 834us/step
1561/1561 ━━━━━━━━━━━━━━━━━━━━ 2s 988us/step
521/521 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

Fold 2/5 for arousal_quartile:
Epoch 1/2
116133/116133 ━━━━━━━━━━━━━━━━━━━━ 131s 1ms/step - loss: 1.7831
Epoch 2/2
116133/116133 ━━━━━━━━━━━━━━━━━━━━ 137s 1ms/step - loss: 0.6009
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 2s 824us/step
1561/1561 ━━━━━━━━━━━━━━━━━━━━ 1s 791us/step
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step

Fold 3/5 for arousal_quartile:
Epoch 1/2
101300/101300 ━━━━━━━━━━━━━━━━━━━━ 121s 1ms/step - loss: 0.4581
Epoch 2/2
101300/101300 ━━━━━━━━━━━━━━━━━━━━ 110s 1ms/step - loss: 0.4320
3629/3629 ━━━━━━━━━━━━━━━━━━━━ 3s 860us/step
2588/2588 ━━━━━━━━━━━━━━━━━━━━ 2s 829us/step
1541/1541 ━━━━━━━━━━━━━━━━━━━━ 1s 880us/step
514/514 ━━━━━━━━━━━━━━━━━

- Evaluation for valence

In [4]:
base_dir = 'RECOLA Ranking Algorithms'

def pairwise_transformation(X, Y, groups):
    transformed_data = []
    transformed_labels = []
    transformed_groups = []
    for i in range(len(X)):
        for j in range(i+1, len(X)):
            xi, xj = X[i], X[j]
            yi, yj = Y[i], Y[j]
            if np.abs(yi - yj) > 0.1:
                pair1 = np.concatenate([xi, xj])
                pair2 = np.concatenate([xj, xi])
                transformed_data.append(pair1)
                transformed_data.append(pair2)

                transformed_groups.append(groups[i])
                transformed_groups.append(groups[i])

                if yi > yj:
                    transformed_labels.append(1)
                    transformed_labels.append(0)
                else:
                    transformed_labels.append(0)
                    transformed_labels.append(1)

    return np.array(transformed_data), np.array(transformed_labels), np.array(transformed_groups)

class RankNet(Model):
    def __init__(self):
        super().__init__()
        self.dense = [layers.Dense(16, activation=leaky_relu), layers.Dense(8, activation=leaky_relu)]
        self.o = layers.Dense(1, activation='linear')
        self.oi_minus_oj = layers.Subtract()

    def call(self, input_tensor):
        half = input_tensor.shape[-1] // 2
        xi, xj = input_tensor[:, :half], input_tensor[:, half:]
        densei = self.dense[0](xi)
        densej = self.dense[0](xj)
        for dense in self.dense[1:]:
            densei = dense(densei)
            densej = dense(densej)
        oi = self.o(densei)
        oj = self.o(densej)
        oij = self.oi_minus_oj([oi, oj])
        output = layers.Activation('sigmoid')(oij)
        return output

input_file = os.path.join(base_dir, 'RECOLA_Intervals_Data', 'ArousalValenceTimeSeries_Quartiles.csv')
df = pd.read_csv(input_file)

def concordance_correlation_coefficient(y_true, y_pred):
    cor = np.corrcoef(y_true, y_pred)[0][1]
    mean_true, mean_pred = np.mean(y_true), np.mean(y_pred)
    var_true, var_pred = np.var(y_true), np.var(y_pred)
    sd_true, sd_pred = np.std(y_true), np.std(y_pred)
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred)**2
    ccc = numerator / denominator
    return ccc

def pearson_correlation_coefficient(y_true, y_pred):
    pcc, _ = pearsonr(y_true, y_pred)
    return pcc

def kendalls_tau_coefficient(y_true, y_pred):
    tau, _ = kendalltau(y_true, y_pred)
    return tau

def evaluate_ranknet_performance(ranknet_model, X_transformed, Y, groups):
    evaluation_results = []
    unique_groups = np.unique(groups)
    for group_id in unique_groups:
        idx = groups == group_id
        group_features = X_transformed[idx]
        group_labels = Y[idx]

        mean_features = group_features.mean(axis=0)
        normalized_features = group_features - mean_features
   
        predicted_scores = ranknet_model.predict(normalized_features).flatten()
   
        ccc_value = concordance_correlation_coefficient(group_labels, predicted_scores)
        pcc_value = pearson_correlation_coefficient(group_labels, predicted_scores)
        kendall_tau_value = kendalls_tau_coefficient(group_labels, predicted_scores)
   
        evaluation_results.append({'GroupID': group_id, 'CCC': ccc_value, 'PCC': pcc_value, 'KendallTau': kendall_tau_value})
    return pd.DataFrame(evaluation_results)

evaluation_results = []  

# Initialize the RankNet model
ranknet_model = RankNet()
ranknet_model.compile(optimizer='adam', loss='binary_crossentropy')

excluded_features = [
        'VIDEO_40_LLD_AU1', 'VIDEO_40_LLD_AU2', 'VIDEO_40_LLD_AU4', 'VIDEO_40_LLD_AU5', 'VIDEO_40_LLD_AU6', 'VIDEO_40_LLD_AU7', 'VIDEO_40_LLD_AU9', 'VIDEO_40_LLD_AU11', 'VIDEO_40_LLD_AU12', 'VIDEO_40_LLD_AU15',
        'VIDEO_40_LLD_AU17', 'VIDEO_40_LLD_AU20', 'VIDEO_40_LLD_AU23', 'VIDEO_40_LLD_AU24', 'VIDEO_40_LLD_AU25', 'VIDEO_40_LLD_Yaw', 'VIDEO_40_LLD_Pitch', 'VIDEO_40_LLD_Roll', 'VIDEO_40_LLD_Opt_mean',
        'VIDEO_40_LLD_Opt_std', 'VIDEO_40_LLD_AU1_delta', 'VIDEO_40_LLD_AU2_delta', 'VIDEO_40_LLD_AU4_delta', 'VIDEO_40_LLD_AU5_delta', 'VIDEO_40_LLD_AU6_delta', 'VIDEO_40_LLD_AU7_delta', 'VIDEO_40_LLD_AU9_delta',
        'VIDEO_40_LLD_AU11_delta', 'VIDEO_40_LLD_AU12_delta', 'VIDEO_40_LLD_AU15_delta', 'VIDEO_40_LLD_AU17_delta', 'VIDEO_40_LLD_AU20_delta', 'VIDEO_40_LLD_AU23_delta', 'VIDEO_40_LLD_AU24_delta', 'VIDEO_40_LLD_AU25_delta',
        'VIDEO_40_LLD_Yaw_delta', 'VIDEO_40_LLD_Pitch_delta', 'VIDEO_40_LLD_Roll_delta', 'VIDEO_40_LLD_Opt_mean_delta', 'VIDEO_40_LLD_Opt_std_delta', 'Face_detection_probability', 'ECG_54_LLD_ECG_HR', 'ECG_54_LLD_ECG_HRV',
        'ECG_54_LLD_ECG_zcr', 'ECG_54_LLD_ECG_FFT_1', 'ECG_54_LLD_ECG_FFT_2', 'ECG_54_LLD_ECG_FFT_3', 'ECG_54_LLD_ECG_FFT_4', 'ECG_54_LLD_ECG_FFT_5', 'ECG_54_LLD_ECG_FFT_6', 'ECG_54_LLD_ECG_FFT_7', 'ECG_54_LLD_ECG_FFT_8',
        'ECG_54_LLD_ECG_FFT_9', 'ECG_54_LLD_ECG_FFT_10', 'ECG_54_LLD_ECG_FFT_11', 'ECG_54_LLD_ECG_FFT_12', 'ECG_54_LLD_ECG_FFT_entropy', 'ECG_54_LLD_ECG_FFT_mean_frequency', 'ECG_54_LLD_ECG_FFT_slope', 'ECG_54_LLD_ECG_mean',
        'ECG_54_LLD_ECG_std', 'ECG_54_LLD_ECG_kurtosis', 'ECG_54_LLD_ECG_skewness', 'ECG_54_LLD_ECG_NSImn', 'ECG_54_LLD_ECG_NLDmn', 'ECG_54_LLD_ECG_VLF', 'ECG_54_LLD_ECG_LF', 'ECG_54_LLD_ECG_HF', 'ECG_54_LLD_ECG_LFHF', 'ECG_54_LLD_ECG_zcr_delta',
        'ECG_54_LLD_ECG_FFT_1_delta', 'ECG_54_LLD_ECG_FFT_2_delta', 'ECG_54_LLD_ECG_FFT_3_delta', 'ECG_54_LLD_ECG_FFT_4_delta', 'ECG_54_LLD_ECG_FFT_5_delta', 'ECG_54_LLD_ECG_FFT_6_delta', 'ECG_54_LLD_ECG_FFT_7_delta', 'ECG_54_LLD_ECG_FFT_8_delta',
        'ECG_54_LLD_ECG_FFT_9_delta', 'ECG_54_LLD_ECG_FFT_10_delta', 'ECG_54_LLD_ECG_FFT_11_delta', 'ECG_54_LLD_ECG_FFT_12_delta', 'ECG_54_LLD_ECG_FFT_entropy_delta', 'ECG_54_LLD_ECG_FFT_mean_frequency_delta', 'ECG_54_LLD_ECG_FFT_slope_delta', 'ECG_54_LLD_ECG_mean_delta',
        'ECG_54_LLD_ECG_std_delta', 'ECG_54_LLD_ECG_kurtosis_delta', 'ECG_54_LLD_ECG_skewness_delta', 'ECG_54_LLD_ECG_NSImn_delta', 'ECG_54_LLD_ECG_NLDmn_delta', 'ECG_54_LLD_ECG_VLF_delta', 'ECG_54_LLD_ECG_LF_delta', 'ECG_54_LLD_ECG_HF_delta', 'ECG_54_LLD_ECG_LFHF_delta',
        'EDA_62_LLD_time_code', 'EDA_62_LLD_EDA_slope', 'EDA_62_LLD_EDA_std', 'EDA_62_LLD_SCR_FFT_entropy', 'EDA_62_LLD_SCR_FFT_mean_frequency', 'EDA_62_LLD_EDA_mean', 'EDA_62_LLD_EDA_meanD', 'EDA_62_LLD_EDA_meanDneg', 'EDA_62_LLD_EDA_prop', 'EDA_62_LLD_EDA_Xbound', 'EDA_62_LLD_EDA_kurtosis',
        'EDA_62_LLD_EDA_skewness', 'EDA_62_LLD_EDA_NSImn', 'EDA_62_LLD_EDA_NLDmn', 'EDA_62_LLD_SCL_mean', 'EDA_62_LLD_SCL_meanD', 'EDA_62_LLD_SCL_meanDneg', 'EDA_62_LLD_SCL_prop', 'EDA_62_LLD_SCL_Xbound', 'EDA_62_LLD_SCL_kurtosis', 'EDA_62_LLD_SCL_skewness', 'EDA_62_LLD_SCL_NSImn',
        'EDA_62_LLD_SCL_NLDmn', 'EDA_62_LLD_SCR_mean', 'EDA_62_LLD_SCR_meanD', 'EDA_62_LLD_SCR_meanDneg', 'EDA_62_LLD_SCR_prop', 'EDA_62_LLD_SCR_Xbound', 'EDA_62_LLD_SCR_kurtosis', 'EDA_62_LLD_SCR_skewness', 'EDA_62_LLD_SCR_NSImn', 'EDA_62_LLD_SCR_NLDmn', 'EDA_62_LLD_EDA_slope_delta',
        'EDA_62_LLD_EDA_std_delta', 'EDA_62_LLD_SCR_FFT_entropy_delta', 'EDA_62_LLD_SCR_FFT_mean_frequency_delta', 'EDA_62_LLD_EDA_mean_delta', 'EDA_62_LLD_EDA_meanD_delta', 'EDA_62_LLD_EDA_meanDneg_delta', 'EDA_62_LLD_EDA_prop_delta', 'EDA_62_LLD_EDA_Xbound_delta', 'EDA_62_LLD_EDA_kurtosis_delta',
        'EDA_62_LLD_EDA_skewness_delta', 'EDA_62_LLD_EDA_NSImn_delta', 'EDA_62_LLD_EDA_NLDmn_delta', 'EDA_62_LLD_SCL_mean_delta', 'EDA_62_LLD_SCL_meanD_delta', 'EDA_62_LLD_SCL_meanDneg_delta', 'EDA_62_LLD_SCL_prop_delta', 'EDA_62_LLD_SCL_Xbound_delta', 'EDA_62_LLD_SCL_kurtosis_delta', 'EDA_62_LLD_SCL_skewness_delta',
        'EDA_62_LLD_SCL_NSImn_delta', 'EDA_62_LLD_SCL_NLDmn_delta', 'EDA_62_LLD_SCR_mean_delta', 'EDA_62_LLD_SCR_meanD_delta', 'EDA_62_LLD_SCR_meanDneg_delta', 'EDA_62_LLD_SCR_prop_delta', 'EDA_62_LLD_SCR_Xbound_delta', 'EDA_62_LLD_SCR_kurtosis_delta', 'EDA_62_LLD_SCR_skewness_delta', 'EDA_62_LLD_SCR_NSImn_delta', 'EDA_62_LLD_SCR_NLDmn_delta'

    ]

feature_columns = [col for col in df.columns if col not in excluded_features + ['participant_id','median_arousal', 'median_valence', 'time_window', 'arousal_quartile', 'valence_quartile']]

targets = ['valence_quartile']

# Initialize GroupKFold
group_kfold = GroupKFold(n_splits=5)

# Process for valence_quartile
for target in targets:
    print(f"\nProcessing for {target}:")
    X = df[feature_columns].values
    Y = df[target].values
    groups = df['participant_id'].values
    for fold, (train_index, test_index) in enumerate(group_kfold.split(X, Y, groups=groups)):
        print(f"\nFold {fold+1}/{group_kfold.n_splits} for {target}:")
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        groups_train, groups_test = groups[train_index], groups[test_index]

        # Transform both training and testing data along with groups
        X_train_transformed, Y_train_transformed, groups_train_transformed = pairwise_transformation(X_train, Y_train, groups_train)
        X_test_transformed, Y_test_transformed, groups_test_transformed = pairwise_transformation(X_test, Y_test, groups_test)

        ranknet_model.fit(X_train_transformed, Y_train_transformed, epochs=2, verbose=1)

        # Insert evaluation after predictions
        evaluation_df = evaluate_ranknet_performance(ranknet_model, X_test_transformed, Y_test_transformed, groups_test_transformed)
        evaluation_results.append(evaluation_df)
        del X_train_transformed, Y_train_transformed
        del X_test_transformed, Y_test_transformed     
        gc.collect()
        
        
# Combine and save evaluation results
combined_results_df = pd.concat(evaluation_results, ignore_index=True)

combined_evaluation_results_file = os.path.join('RECOLA Ranking Algorithms','Evaluation','RankNET', 'ranknet_evaluation_results_valence.csv')
combined_results_df.to_csv(combined_evaluation_results_file, index=False)

print("Individual performance evaluation valence results saved.")




Processing for valence_quartile:

Fold 1/5 for valence_quartile:
Epoch 1/2
116133/116133 ━━━━━━━━━━━━━━━━━━━━ 150s 1ms/step - loss: 493.8092
Epoch 2/2
116133/116133 ━━━━━━━━━━━━━━━━━━━━ 142s 1ms/step - loss: 0.6754
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 3s 970us/step
1561/1561 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
521/521 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

Fold 2/5 for valence_quartile:
Epoch 1/2
116133/116133 ━━━━━━━━━━━━━━━━━━━━ 142s 1ms/step - loss: 0.6148
Epoch 2/2
116133/116133 ━━━━━━━━━━━━━━━━━━━━ 141s 1ms/step - loss: 0.5528
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 2s 877us/step
1561/1561 ━━━━━━━━━━━━━━━━━━━━ 1s 909us/step
521/521 ━━━━━━━━━━━━━━━━━━━━ 1s 977us/step

Fold 3/5 for valence_quartile:
Epoch 1/2
101300/101300 ━━━━━━━━━━━━━━━━━━━━ 126s 1ms/step - loss: 0.4881
Epoch 2/2
101300/101300 ━━━━━━━━━━━━━━━━━━━━ 112s 1ms/step - loss: 0.4579
3629/3629 ━━━━━━━━━━━━━━━━━━━━ 3s 833us/step
2588/2588 ━━━━━━━━━━━━━━━━━━━━ 2s 848us/step
1541/1541 ━━━━━━━━━━━━━━━━━━━━ 1s 764us/step
514/514 ━━━━━━━━━━━━━━━━━━━

In [4]:
base_dir = 'RECOLA Ranking Algorithms'

def pairwise_transformation(X, Y, groups):
    transformed_data = []
    transformed_labels = []
    transformed_groups = []
    for i in range(len(X)):
        for j in range(i+1, len(X)):
            xi, xj = X[i], X[j]
            yi, yj = Y[i], Y[j]
            if np.abs(yi - yj) > 0.1:
                pair1 = np.concatenate([xi, xj])
                pair2 = np.concatenate([xj, xi])
                transformed_data.append(pair1)
                transformed_data.append(pair2)

                transformed_groups.append(groups[i])
                transformed_groups.append(groups[i])

                if yi > yj:
                    transformed_labels.append(1)
                    transformed_labels.append(0)
                else:
                    transformed_labels.append(0)
                    transformed_labels.append(1)

    return np.array(transformed_data), np.array(transformed_labels), np.array(transformed_groups)

class RankNet(Model):
    def __init__(self):
        super().__init__()
        self.dense = [layers.Dense(16, activation=leaky_relu), layers.Dense(8, activation=leaky_relu)]
        self.o = layers.Dense(1, activation='linear')
        self.oi_minus_oj = layers.Subtract()

    def call(self, input_tensor):
        half = input_tensor.shape[-1] // 2
        xi, xj = input_tensor[:, :half], input_tensor[:, half:]
        densei = self.dense[0](xi)
        densej = self.dense[0](xj)
        for dense in self.dense[1:]:
            densei = dense(densei)
            densej = dense(densej)
        oi = self.o(densei)
        oj = self.o(densej)
        oij = self.oi_minus_oj([oi, oj])
        output = layers.Activation('sigmoid')(oij)
        return output

input_file = os.path.join(base_dir, 'RECOLA_Intervals_Data', 'ArousalValenceTimeSeries_Quartiles.csv')
df = pd.read_csv(input_file)

def concordance_correlation_coefficient(y_true, y_pred):
    cor = np.corrcoef(y_true, y_pred)[0][1]
    mean_true, mean_pred = np.mean(y_true), np.mean(y_pred)
    var_true, var_pred = np.var(y_true), np.var(y_pred)
    sd_true, sd_pred = np.std(y_true), np.std(y_pred)
    numerator = 2 * cor * sd_true * sd_pred
    denominator = var_true + var_pred + (mean_true - mean_pred)**2
    ccc = numerator / denominator
    return ccc

def pearson_correlation_coefficient(y_true, y_pred):
    pcc, _ = pearsonr(y_true, y_pred)
    return pcc

def kendalls_tau_coefficient(y_true, y_pred):
    tau, _ = kendalltau(y_true, y_pred)
    return tau

def evaluate_ranknet_performance(ranknet_model, X_transformed, Y, groups):
    evaluation_results = []
    unique_groups = np.unique(groups)
    for group_id in unique_groups:
        idx = groups == group_id
        group_features = X_transformed[idx]
        group_labels = Y[idx]

        mean_features = group_features.mean(axis=0)
        normalized_features = group_features - mean_features
   
        predicted_scores = ranknet_model.predict(normalized_features).flatten()
   
        ccc_value = concordance_correlation_coefficient(group_labels, predicted_scores)
        pcc_value = pearson_correlation_coefficient(group_labels, predicted_scores)
        kendall_tau_value = kendalls_tau_coefficient(group_labels, predicted_scores)
   
        evaluation_results.append({'GroupID': group_id, 'CCC': ccc_value, 'PCC': pcc_value, 'KendallTau': kendall_tau_value})
    return pd.DataFrame(evaluation_results)

evaluation_results = []  

# Initialize the RankNet model
ranknet_model = RankNet()
ranknet_model.compile(optimizer='adam', loss='binary_crossentropy')

excluded_features = [
        'VIDEO_40_LLD_AU1', 'VIDEO_40_LLD_AU2', 'VIDEO_40_LLD_AU4', 'VIDEO_40_LLD_AU5', 'VIDEO_40_LLD_AU6', 'VIDEO_40_LLD_AU7', 'VIDEO_40_LLD_AU9', 'VIDEO_40_LLD_AU11', 'VIDEO_40_LLD_AU12', 'VIDEO_40_LLD_AU15',
        'VIDEO_40_LLD_AU17', 'VIDEO_40_LLD_AU20', 'VIDEO_40_LLD_AU23', 'VIDEO_40_LLD_AU24', 'VIDEO_40_LLD_AU25', 'VIDEO_40_LLD_Yaw', 'VIDEO_40_LLD_Pitch', 'VIDEO_40_LLD_Roll', 'VIDEO_40_LLD_Opt_mean',
        'VIDEO_40_LLD_Opt_std', 'VIDEO_40_LLD_AU1_delta', 'VIDEO_40_LLD_AU2_delta', 'VIDEO_40_LLD_AU4_delta', 'VIDEO_40_LLD_AU5_delta', 'VIDEO_40_LLD_AU6_delta', 'VIDEO_40_LLD_AU7_delta', 'VIDEO_40_LLD_AU9_delta',
        'VIDEO_40_LLD_AU11_delta', 'VIDEO_40_LLD_AU12_delta', 'VIDEO_40_LLD_AU15_delta', 'VIDEO_40_LLD_AU17_delta', 'VIDEO_40_LLD_AU20_delta', 'VIDEO_40_LLD_AU23_delta', 'VIDEO_40_LLD_AU24_delta', 'VIDEO_40_LLD_AU25_delta',
        'VIDEO_40_LLD_Yaw_delta', 'VIDEO_40_LLD_Pitch_delta', 'VIDEO_40_LLD_Roll_delta', 'VIDEO_40_LLD_Opt_mean_delta', 'VIDEO_40_LLD_Opt_std_delta', 'Face_detection_probability', 'ECG_54_LLD_ECG_HR', 'ECG_54_LLD_ECG_HRV',
        'ECG_54_LLD_ECG_zcr', 'ECG_54_LLD_ECG_FFT_1', 'ECG_54_LLD_ECG_FFT_2', 'ECG_54_LLD_ECG_FFT_3', 'ECG_54_LLD_ECG_FFT_4', 'ECG_54_LLD_ECG_FFT_5', 'ECG_54_LLD_ECG_FFT_6', 'ECG_54_LLD_ECG_FFT_7', 'ECG_54_LLD_ECG_FFT_8',
        'ECG_54_LLD_ECG_FFT_9', 'ECG_54_LLD_ECG_FFT_10', 'ECG_54_LLD_ECG_FFT_11', 'ECG_54_LLD_ECG_FFT_12', 'ECG_54_LLD_ECG_FFT_entropy', 'ECG_54_LLD_ECG_FFT_mean_frequency', 'ECG_54_LLD_ECG_FFT_slope', 'ECG_54_LLD_ECG_mean',
        'ECG_54_LLD_ECG_std', 'ECG_54_LLD_ECG_kurtosis', 'ECG_54_LLD_ECG_skewness', 'ECG_54_LLD_ECG_NSImn', 'ECG_54_LLD_ECG_NLDmn', 'ECG_54_LLD_ECG_VLF', 'ECG_54_LLD_ECG_LF', 'ECG_54_LLD_ECG_HF', 'ECG_54_LLD_ECG_LFHF', 'ECG_54_LLD_ECG_zcr_delta',
        'ECG_54_LLD_ECG_FFT_1_delta', 'ECG_54_LLD_ECG_FFT_2_delta', 'ECG_54_LLD_ECG_FFT_3_delta', 'ECG_54_LLD_ECG_FFT_4_delta', 'ECG_54_LLD_ECG_FFT_5_delta', 'ECG_54_LLD_ECG_FFT_6_delta', 'ECG_54_LLD_ECG_FFT_7_delta', 'ECG_54_LLD_ECG_FFT_8_delta',
        'ECG_54_LLD_ECG_FFT_9_delta', 'ECG_54_LLD_ECG_FFT_10_delta', 'ECG_54_LLD_ECG_FFT_11_delta', 'ECG_54_LLD_ECG_FFT_12_delta', 'ECG_54_LLD_ECG_FFT_entropy_delta', 'ECG_54_LLD_ECG_FFT_mean_frequency_delta', 'ECG_54_LLD_ECG_FFT_slope_delta', 'ECG_54_LLD_ECG_mean_delta',
        'ECG_54_LLD_ECG_std_delta', 'ECG_54_LLD_ECG_kurtosis_delta', 'ECG_54_LLD_ECG_skewness_delta', 'ECG_54_LLD_ECG_NSImn_delta', 'ECG_54_LLD_ECG_NLDmn_delta', 'ECG_54_LLD_ECG_VLF_delta', 'ECG_54_LLD_ECG_LF_delta', 'ECG_54_LLD_ECG_HF_delta', 'ECG_54_LLD_ECG_LFHF_delta',
        'EDA_62_LLD_time_code', 'EDA_62_LLD_EDA_slope', 'EDA_62_LLD_EDA_std', 'EDA_62_LLD_SCR_FFT_entropy', 'EDA_62_LLD_SCR_FFT_mean_frequency', 'EDA_62_LLD_EDA_mean', 'EDA_62_LLD_EDA_meanD', 'EDA_62_LLD_EDA_meanDneg', 'EDA_62_LLD_EDA_prop', 'EDA_62_LLD_EDA_Xbound', 'EDA_62_LLD_EDA_kurtosis',
        'EDA_62_LLD_EDA_skewness', 'EDA_62_LLD_EDA_NSImn', 'EDA_62_LLD_EDA_NLDmn', 'EDA_62_LLD_SCL_mean', 'EDA_62_LLD_SCL_meanD', 'EDA_62_LLD_SCL_meanDneg', 'EDA_62_LLD_SCL_prop', 'EDA_62_LLD_SCL_Xbound', 'EDA_62_LLD_SCL_kurtosis', 'EDA_62_LLD_SCL_skewness', 'EDA_62_LLD_SCL_NSImn',
        'EDA_62_LLD_SCL_NLDmn', 'EDA_62_LLD_SCR_mean', 'EDA_62_LLD_SCR_meanD', 'EDA_62_LLD_SCR_meanDneg', 'EDA_62_LLD_SCR_prop', 'EDA_62_LLD_SCR_Xbound', 'EDA_62_LLD_SCR_kurtosis', 'EDA_62_LLD_SCR_skewness', 'EDA_62_LLD_SCR_NSImn', 'EDA_62_LLD_SCR_NLDmn', 'EDA_62_LLD_EDA_slope_delta',
        'EDA_62_LLD_EDA_std_delta', 'EDA_62_LLD_SCR_FFT_entropy_delta', 'EDA_62_LLD_SCR_FFT_mean_frequency_delta', 'EDA_62_LLD_EDA_mean_delta', 'EDA_62_LLD_EDA_meanD_delta', 'EDA_62_LLD_EDA_meanDneg_delta', 'EDA_62_LLD_EDA_prop_delta', 'EDA_62_LLD_EDA_Xbound_delta', 'EDA_62_LLD_EDA_kurtosis_delta',
        'EDA_62_LLD_EDA_skewness_delta', 'EDA_62_LLD_EDA_NSImn_delta', 'EDA_62_LLD_EDA_NLDmn_delta', 'EDA_62_LLD_SCL_mean_delta', 'EDA_62_LLD_SCL_meanD_delta', 'EDA_62_LLD_SCL_meanDneg_delta', 'EDA_62_LLD_SCL_prop_delta', 'EDA_62_LLD_SCL_Xbound_delta', 'EDA_62_LLD_SCL_kurtosis_delta', 'EDA_62_LLD_SCL_skewness_delta',
        'EDA_62_LLD_SCL_NSImn_delta', 'EDA_62_LLD_SCL_NLDmn_delta', 'EDA_62_LLD_SCR_mean_delta', 'EDA_62_LLD_SCR_meanD_delta', 'EDA_62_LLD_SCR_meanDneg_delta', 'EDA_62_LLD_SCR_prop_delta', 'EDA_62_LLD_SCR_Xbound_delta', 'EDA_62_LLD_SCR_kurtosis_delta', 'EDA_62_LLD_SCR_skewness_delta', 'EDA_62_LLD_SCR_NSImn_delta', 'EDA_62_LLD_SCR_NLDmn_delta'

    ]

#feature_columns = [col for col in df.columns if col not in excluded_features + ['participant_id','median_arousal', 'median_valence', 'time_window', 'arousal_quartile', 'valence_quartile']]
feature_columns = [col for col in df.columns if col not in ['participant_id',
                                                     'median_arousal',
                                                     'median_valence',
                                                     'time_window',
                                                     'time in seconds',
                                                     'FM1 _x', 'FM2 _x', 'FM3 _x', 'FF1 _x', 'FF2 _x', 'FF3_x',
                                                     'FM1 _y', 'FM2 _y', 'FM3 _y', 'FF1 _y', 'FF2 _y', 'FF3_y']]

targets = ['valence_quartile']

# Initialize GroupKFold
group_kfold = GroupKFold(n_splits=5)

# Process for valence_quartile
for target in targets:
    print(f"\nProcessing for {target}:")
    X = df[feature_columns].values
    Y = df[target].values
    groups = df['participant_id'].values
    for fold, (train_index, test_index) in enumerate(group_kfold.split(X, Y, groups=groups)):
        print(f"\nFold {fold+1}/{group_kfold.n_splits} for {target}:")
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        groups_train, groups_test = groups[train_index], groups[test_index]

        # Transform both training and testing data along with groups
        X_train_transformed, Y_train_transformed, groups_train_transformed = pairwise_transformation(X_train, Y_train, groups_train)
        X_test_transformed, Y_test_transformed, groups_test_transformed = pairwise_transformation(X_test, Y_test, groups_test)

        ranknet_model.fit(X_train_transformed, Y_train_transformed, epochs=2, verbose=1)

        # Insert evaluation after predictions
        evaluation_df = evaluate_ranknet_performance(ranknet_model, X_test_transformed, Y_test_transformed, groups_test_transformed)
        evaluation_results.append(evaluation_df)
        del X_train_transformed, Y_train_transformed
        del X_test_transformed, Y_test_transformed     
        gc.collect()
        
        
# Combine and save evaluation results
combined_results_df = pd.concat(evaluation_results, ignore_index=True)

combined_evaluation_results_file = os.path.join( 'ranknet_evaluation_results_valence.csv')
combined_results_df.to_csv(combined_evaluation_results_file, index=False)

print("Individual performance evaluation valence results saved.")




Processing for valence_quartile:

Fold 1/5 for valence_quartile:
